<a href="https://colab.research.google.com/github/chenchongsong/udacity-cs344-colab/blob/main/notebook/udacity_cs344_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Homework 5 for Udacity CS344 Course, Intro to Parallel Programming
# clone the code repo,
!git clone https://github.com/chenchongsong/udacity-cs344-colab
!pip install git+git://github.com/depctg/nvcc4jupyter.git

# load cuda plugin
%config NVCCPluginV2.static_dir = True
%config NVCCPluginV2.relative_dir = "udacity-cs344-colab/src/HW5"
%load_ext nvcc_plugin

# change to work directory, generate makefiles
!mkdir udacity-cs344-colab/build
%cd udacity-cs344-colab/build
!cmake ../src

In [ ]:
%%cuda --name student.cu

/* Udacity HW5
   Histogramming for Speed

   The goal of this assignment is compute a histogram
   as fast as possible.  We have simplified the problem as much as
   possible to allow you to focus solely on the histogramming algorithm.

   The input values that you need to histogram are already the exact
   bins that need to be updated.  This is unlike in HW3 where you needed
   to compute the range of the data and then do:
   bin = (val - valMin) / valRange to determine the bin.

   Here the bin is just:
   bin = val

   so the serial histogram calculation looks like:
   for (i = 0; i < numElems; ++i)
     histo[val[i]]++;

   That's it!  Your job is to make it run as fast as possible!

   The values are normally distributed - you may take
   advantage of this fact in your implementation.

*/


#include "utils.h"

#define HISTOGRAM1024_BIN_COUNT 1024
typedef unsigned int uint;

#define LOG2_WARP_SIZE 5U
#define WARP_SIZE 32U

//Warps == subhistograms per threadblock, WARP_COUNT must be <= 12 due to shared memory limit
#define WARP_COUNT 8

#define HISTOGRAM1024_THREADBLOCK_SIZE (WARP_COUNT * WARP_SIZE)

//Shared memory per threadblock
#define HISTOGRAM1024_THREADBLOCK_MEMORY (WARP_COUNT * HISTOGRAM1024_BIN_COUNT)

#define UMUL(a, b) ( (a) * (b) )
#define UMAD(a, b, c) ( UMUL((a), (b)) + (c) )

inline __device__ void addWarpHist(uint *s_WarpHist, uint data) {
    atomicAdd(s_WarpHist + data, 1);
}

__global__ void histogram1024Kernel(uint *d_PartialHistograms, uint *d_Data, uint dataCount) {
    //Per-warp subhistogram storage
    __shared__ uint s_Hist[HISTOGRAM1024_THREADBLOCK_MEMORY];
    uint *s_WarpHist= s_Hist + (threadIdx.x >> LOG2_WARP_SIZE) * HISTOGRAM1024_BIN_COUNT;

    //Clear shared memory storage for current threadblock before processing
#pragma unroll
    for (uint i = 0; i < (HISTOGRAM1024_THREADBLOCK_MEMORY / HISTOGRAM1024_THREADBLOCK_SIZE); i++) {
        s_Hist[threadIdx.x + i * HISTOGRAM1024_THREADBLOCK_SIZE] = 0;
    }
    __syncthreads();
    // till here, ~0.15ms

    //Cycle through the entire data set, update subhistograms for each warp
    for (uint pos = UMAD(blockIdx.x, blockDim.x, threadIdx.x); pos < dataCount; pos += UMUL(blockDim.x, gridDim.x)) {
        uint data = d_Data[pos];
        addWarpHist(s_WarpHist, data);
    }
    // till here, ~2.83ms

    //Merge per-warp histograms into per-block and write to global memory
    __syncthreads();

    for (uint bin = threadIdx.x; bin < HISTOGRAM1024_BIN_COUNT; bin += HISTOGRAM1024_THREADBLOCK_SIZE) {
        uint sum = 0;
        for (uint i = 0; i < WARP_COUNT; i++) {
            sum += s_Hist[bin + i * HISTOGRAM1024_BIN_COUNT];
        }
        d_PartialHistograms[blockIdx.x * HISTOGRAM1024_BIN_COUNT + bin] = sum;
    }
}

////////////////////////////////////////////////////////////////////////////////
// Merge histogram1024() output
// Run one threadblock per bin; each threadblock adds up the same bin counter
// from every partial histogram. Reads are uncoalesced, but mergeHistogram1024
// takes only a fraction of total processing time
////////////////////////////////////////////////////////////////////////////////
#define MERGE_THREADBLOCK_SIZE 256

__global__ void mergeHistogram1024Kernel(
    uint *d_Histogram,
    uint *d_PartialHistograms,
    uint histogramCount
) {
    uint sum = 0;

    for (uint i = threadIdx.x; i < histogramCount; i += MERGE_THREADBLOCK_SIZE) {
        sum += d_PartialHistograms[blockIdx.x + i * HISTOGRAM1024_BIN_COUNT];
    }

    __shared__ uint data[MERGE_THREADBLOCK_SIZE];
    data[threadIdx.x] = sum;

    // reduce
    for (uint stride = MERGE_THREADBLOCK_SIZE / 2; stride > 0; stride >>= 1) {
        __syncthreads();
        if (threadIdx.x < stride) {
            data[threadIdx.x] += data[threadIdx.x + stride];
        }
    }
    if (threadIdx.x == 0) {
        d_Histogram[blockIdx.x] = data[0];
    }
}

////////////////////////////////////////////////////////////////////////////////
// Host interface to GPU histogram
////////////////////////////////////////////////////////////////////////////////
//histogram1024kernel() intermediate results buffer
static const uint PARTIAL_HISTOGRAM1024_COUNT = 256;
static uint *d_PartialHistograms;

void computeHistogram(const uint* const d_vals, //INPUT
                      uint* const d_histo,      //OUTPUT
                      const uint numBins,
                      const uint numElems) {
  //numBins: 1024, numElems: 10240000

  checkCudaErrors(cudaMalloc((void**)&d_PartialHistograms, PARTIAL_HISTOGRAM1024_COUNT * HISTOGRAM1024_BIN_COUNT * sizeof(uint)));
  
  // 固定thread block个数，然后每个thread block都会算出一个partial histogram
  histogram1024Kernel<<<PARTIAL_HISTOGRAM1024_COUNT, HISTOGRAM1024_THREADBLOCK_SIZE>>>(
      d_PartialHistograms,
      (uint*)d_vals,
      numElems
  );  // ~3ms
  checkCudaErrors(cudaGetLastError());

  // 每个thread block处理所有partial histogram中的同一位
  mergeHistogram1024Kernel<<<HISTOGRAM1024_BIN_COUNT, MERGE_THREADBLOCK_SIZE>>>(
      d_histo,
      d_PartialHistograms,
      PARTIAL_HISTOGRAM1024_COUNT
  );  // ~0.15ms
  checkCudaErrors(cudaGetLastError());

  checkCudaErrors(cudaFree(d_PartialHistograms));
}

In [ ]:
# make the cuda project
!nvidia-smi
!make HW5
print("\n====== RESULT OF HW5 =======\n")
!bin/HW5